In [1]:
# pip install PyMuPDF pytesseract langchain crewai
# pip install faiss-cpu

In [ ]:
from crewai import Agent, Task, Crew, Process
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from crewai_tools import BaseTool
import fitz  # PyMuPDF
import pytesseract
from PIL import Image

In [17]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_ENDPOINT"] = ""

In [3]:
class PDFOCRReaderTool(BaseTool):
    name: str = "PDFOCRReaderTool"
    description: str = "Ferramenta para ler e extrair texto de arquivos PDF usando OCR."

    def _run(self, file_path: str) -> str:
        # Abrir o arquivo PDF
        doc = fitz.open(file_path)
        text = ""
        # Extrair texto de cada página usando OCR
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            pix = page.get_pixmap()
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            text += pytesseract.image_to_string(img)
        return text


2024-05-23 10:35:10,023 [embedchain] [INFO] Swapped std-lib sqlite3 with pysqlite3 for ChromaDb compatibility. Your original version was 3.31.1.


In [4]:
class RAGTool(BaseTool):
    name: str = "RAGTool"
    description: str = "Ferramenta para realizar RAG (Retrieve-and-Generate) em texto."

    def _run(self, text: str, queries: list) -> dict:
        embeddings = OpenAIEmbeddings()
        store = FAISS.from_texts([text], embeddings)
        llm = OpenAI()
        chain = load_qa_chain(llm, chain_type="stuff")

        results = {}
        for query in queries:
            docs = store.similarity_search(query)
            result = chain.run(input_documents=docs, question=query)
            results[query] = result

        return results


In [22]:
perguntador = Agent(
    role='Perguntador',
    goal='Formular perguntas a partir das chaves do dicionário {input_dict}',
    verbose=True,
    memory=True,
    backstory=(
        "Você é um especialista em transformar chaves de dicionários"
        "em perguntas claras e concisas para pesquisa."
    ),
    tools=[],
    allow_delegation=False
)

In [23]:

# Criar as ferramentas PDFOCRReaderTool e RAGTool
pdf_ocr_reader_tool = PDFOCRReaderTool()
rag_tool = RAGTool()

# Criar um agente que utilizará as ferramentas
pesquisador_pdf = Agent(
    role='Pesquisador de PDF',
    goal='Extrair e analisar informações de documentos PDF',
    verbose=True,
    memory=True,
    backstory=(
        "Você é um especialista em leitura e análise de documentos, "
        "capaz de extrair informações importantes de arquivos PDF utilizando ferrament RAG"
    ),
    tools=[pdf_ocr_reader_tool, rag_tool],
    allow_delegation=True
)
formulacao_perguntas = Task(
    description=(
        "Transforme as chaves do dicionário em perguntas."
        "Cada chave deve ser transformada em uma pergunta clara e direta."
    ),
    expected_output='Uma lista de perguntas baseadas nas chaves do dicionário.',
    tools=[],
    agent=perguntador
)

# Criar uma tarefa para o agente pesquisador de PDF
tarefa_extracao = Task(
    description=(
        "Leia o arquivo PDF no path {file_path} e extraia todas as informações relevantes. "
        "Use OCR para extrair texto se necessário. Em seguida, realize uma RAG (Retrieve-and-Generate) "
        "para responder às perguntas fornecidas."
    ),
    expected_output='Um dicionário com as seguintes informações {input_dict}',
    tools=[pdf_ocr_reader_tool, rag_tool],
    agent=pesquisador_pdf,
    # inputs={
    #     'file_path': '{file_path}',
    #     'queries': ['Pergunta 1', 'Pergunta 2', 'Pergunta 3']
    # }
)

# Formar a equipe com o agente configurado
equipe = Crew(
    agents=[perguntador, pesquisador_pdf],
    tasks=[formulacao_perguntas, tarefa_extracao],
    process=Process.sequential  # Execução sequencial das tarefas é o padrão
)


input_dict = {
    "Nome da Empresa": None, 
    "Tipo de Sociedade": None, 
    "Sócio": None, 
    "Endereço da Empresa": None, 
    "Prazo de Duração da Sociedade": None, 
    "Atividade da Empresa": None, 
    "Capital Social": None, 
    "Condições de Transferência de Quotas": None, 
    "Data de Constituição": None, 
    "Status de Registro": None
}

# Iniciar a equipe e realizar a extração e análise
resultado = equipe.kickoff(inputs={
    'file_path': 'docs/664b4a9dff3572d1941cf260.pdf',
    'input_dict': input_dict
})
print(resultado)


2024-05-23 14:25:30,870 - 140103819065152 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed




> Entering new CrewAgentExecutor chain...
Para transformar as chaves do dicionário em perguntas, devo considerar o contexto de cada chave. As chaves parecem ser relacionadas a informações sobre empresas, por isso, vou formular perguntas claras e diretas que se relacionem a este contexto. 

Final Answer:

1. Qual é o nome da empresa?
2. Qual é o tipo de sociedade da empresa?
3. Quem é o sócio da empresa?
4. Qual é o endereço da empresa?
5. Qual é o prazo de duração da sociedade da empresa?
6. Qual é a atividade da empresa?
7. Quanto é o capital social da empresa?
8. Quais são as condições de transferência de quotas da empresa?
9. Quando foi a data de constituição da empresa?
10. Qual é o status de registro da empresa?

> Finished chain.


> Entering new CrewAgentExecutor chain...
Preciso começar lendo o arquivo PDF para conseguir extrair o texto dele. Como o arquivo pode conter imagens ou texto não selecionável, vou usar a ferramenta PDFOCRReaderTool, que utiliza OCR (Optical Characte

In [24]:
a = {'Nome da Empresa': 'UMEGA CONSTRUCOES MODULARES LTDA', 'Tipo de Sociedade': 'Não é possível determinar o tipo de sociedade da empresa com base nas informações fornecidas. É necessário ter acesso aos documentos legais da empresa para confirmar seu tipo de sociedade.', 'Sócio': 'Não é possível determinar quem é o sócio da empresa apenas com as informações fornecidas. Seria necessário ter acesso aos documentos mencionados nas notas para verificar a composição societária da empresa em questão.', 'Endereço da Empresa': 'Rua Atalia Velha, 140, apto. 142-A, na cidade de São Paulo, estado de São Paulo', 'Prazo de Duração da Sociedade': 'Não é possível determinar o prazo de duração da sociedade da empresa com base nas informações fornecidas.', 'Atividade da Empresa': 'Não é possível determinar a atividade da empresa apenas com essas informações. Seria necessário consultar outros documentos ou investigar mais sobre a empresa para obter essa informação.', 'Capital Social': 'Não é possível determinar o capital social da empresa a partir das informações fornecidas. É necessário ter acesso aos documentos mencionados na ficha, como o Contrato Social e o instrumento particular, para obter essa informação.', 'Condições de Transferência de Quotas': 'As condições de transferência de quotas da empresa não estão explicitamente mencionadas no texto. É possível inferir que a transferência de quotas foi feita através da venda do imóvel pelo valor de RS60.000,00, e que os compradores também deram o imóvel em hipoteca para garantir o pagamento de uma dívida de R$43.800,00. No entanto, as condições específicas da transferência de quotas da empresa não estão mencionadas no texto.', 'Data de Constituição': '29 de setembro de 2005', 'Status de Registro': 'A empresa está registrada no 3° Oficial de Registro de Títulos e Documentos e Civil de Pessoa Jurídica, com alteração de denominação social para GNM Empreendimentos e Participações Ltda.'}

In [25]:
import json
json.loads(json.dumps(a))

{'Nome da Empresa': 'UMEGA CONSTRUCOES MODULARES LTDA',
 'Tipo de Sociedade': 'Não é possível determinar o tipo de sociedade da empresa com base nas informações fornecidas. É necessário ter acesso aos documentos legais da empresa para confirmar seu tipo de sociedade.',
 'Sócio': 'Não é possível determinar quem é o sócio da empresa apenas com as informações fornecidas. Seria necessário ter acesso aos documentos mencionados nas notas para verificar a composição societária da empresa em questão.',
 'Endereço da Empresa': 'Rua Atalia Velha, 140, apto. 142-A, na cidade de São Paulo, estado de São Paulo',
 'Prazo de Duração da Sociedade': 'Não é possível determinar o prazo de duração da sociedade da empresa com base nas informações fornecidas.',
 'Atividade da Empresa': 'Não é possível determinar a atividade da empresa apenas com essas informações. Seria necessário consultar outros documentos ou investigar mais sobre a empresa para obter essa informação.',
 'Capital Social': 'Não é possível 

In [ ]:
# from crewai import Agent, Task, Crew, Process

# # Criar as ferramentas PDFOCRReaderTool e RAGTool
# pdf_ocr_reader_tool = PDFOCRReaderTool()
# rag_tool = RAGTool()

# # Criar um agente que utilizará as ferramentas
# pesquisador_pdf = Agent(
#     role='Pesquisador de PDF',
#     goal='Extrair e analisar informações de documentos PDF',
#     verbose=True,
#     memory=True,
#     backstory=(
#         "Você é um especialista em leitura e análise de documentos, "
#         "capaz de extrair informações importantes de arquivos PDF."
#     ),
#     tools=[pdf_ocr_reader_tool, rag_tool],
#     allow_delegation=True
# )

# # Criar uma tarefa para o agente pesquisador de PDF
# class ExtracaoTask(Task):
#     def __init__(self, **kwargs):
#         super().__init__(**kwargs)
#         # self.extracted_text = []

#     def run(self, inputs):
#         file_path = inputs['file_path']
#         queries = inputs.get('queries', [])
        
#         if self.extracted_text is None:
#             self.extracted_text = pdf_ocr_reader_tool._run(file_path)
        
#         if queries:
#             results = rag_tool._run(self.extracted_text, queries)
#             return results
#         return {}

# tarefa_extracao = ExtracaoTask(
#     description=(
#         "Leia o arquivo PDF no path {file_path} e extraia todas as informações relevantes. "
#         "Use OCR para extrair texto se necessário. Em seguida, realize uma RAG (Retrieve-and-Generate) "
#         "para responder às perguntas fornecidas."
#     ),
#     expected_output='Um dicionário com as respostas para as perguntas fornecidas.',
#     tools=[pdf_ocr_reader_tool, rag_tool],
#     agent=pesquisador_pdf,
# )

# # Formar a equipe com o agente configurado
# equipe = Crew(
#     agents=[pesquisador_pdf],
#     tasks=[tarefa_extracao],
#     process=Process.sequential  # Execução sequencial das tarefas é o padrão
# )

# # Primeira execução para extrair texto e responder perguntas iniciais
# resultado1 = equipe.kickoff(inputs={
#     'file_path': 'docs/664b5344fb83d1fb6e1857b8.pdf',
#     'queries': ['Nome da Empresa', 'Tipo de Sociedade']
# })
# print(resultado1)

# # Segunda execução para adicionar mais perguntas sem reprocessar OCR
# resultado2 = equipe.kickoff(inputs={
#     'file_path': 'docs/664b5344fb83d1fb6e1857b8.pdf',
#     'queries': ['Sócio', 'Endereço da Empresa']
# })
# print(resultado2)
